# ĐỒ ÁN CUỐI KỲ: Phân tích tỉ lệ rời bỏ của khách hàng trong BankChurners Dataset
Môn học: CSC17104 – Lập trình cho Khoa học Dữ liệu

Khoa: Công nghệ Thông tin - ĐH KHTN, ĐHQG-HCM

Nhóm 26:
Sinh viên thực hiện:
- 23122035 - Châu Văn Minh Khoa
- 23122046 - Phan Ngọc Quân 

## I. DATA COLLECTION

### Dữ liệu của bạn là về chủ đề gì?


- Ngành ngân hàng chịu áp lực lớn trong việc giữ chân khách hàng thẻ tín dụng; nếu khách đóng tài khoản thì doanh thu từ phí và lãi giảm đáng kể.


- Bộ dữ liệu BankChurners ghi nhận hồ sơ khách hàng cá nhân, thói quen sử dụng thẻ, tương tác dịch vụ và trạng thái rời bỏ (`Attrition_Flag`) để đo lường khả năng ở lại hay rời đi của khách hàng.

### Nguồn dữ liệu
- Nguồn: Kaggle – Credit Card Customers (tệp `BankChurners.csv`, https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers).
- Public date(Kaggle): December 1, 2024

### Quyền sử dụng
- License: CC0-Public Domain, được chia sẻ cho mục đích học tập và nghiên cứu.


- Mục đích sử dụng: phục vụ môn học, tuân thủ điều khoản Kaggle; dữ liệu đã được ẩn danh nhưng vẫn cần tránh suy diễn định danh cá nhân cụ thể.


- Lưu ý đạo đức: các biến tài chính nên được diễn giải có trách nhiệm, không gán nhãn tiêu cực cho nhóm khách hàng cụ thể; khi chia sẻ kết quả cần nêu rõ đây là dữ liệu giả lập/ẩn danh.

### Người ta thu thập dữ liệu như thế nào?
Các bước thực hiện:
- Đăng nhập vào website https://leaps.analyttica.com/home.
- Click on the 'Apply' tab trên góc trên bên trái
- Ở đây, ta có thể tìm thấy nhiều vấn đề của các doanh nghiệp để giải quyết cùng với bộ dataset từ họ.

Ở trên là các bước thực hiện được ghi trên kaggle. Tuy nhiên, do bộ dữ liệu đã khá cũ nên cách trên hiện nay không hoạt động được. Nhóm em có check thử thì thấy trang web đã thay đổi tên miền.

### Mục đích lựa chọn dữ liệu này

- Dataset có đủ dữ liệu để phân tích (10127 dòng, 23 cột) đa dạng biến về nhân khẩu học, lịch sử giao dịch, mức độ tương tác; phù hợp để luyện quy trình EDA, tiền xử lý và xây dựng mô hình phân loại churn.
- Có cả dữ liệu numerical và categorical nên có thể áp dụng nhiều kỹ thuật phân tích
- Dữ liệu thực tế từ người dùng nên có ý nghĩa thực tế
- Bối cảnh ngân hàng quen thuộc với các bài toán khoa học dữ liệu hướng kinh doanh, thuận tiện để trình bày insight định lượng và gợi ý chiến lược.

### Mô tả dữ liệu


- Mỗi dòng: Một khách hàng thẻ tín dụng với trạng thái tiếp tục hay rời bỏ, kèm các đặc trưng nhân khẩu học và hành vi sử dụng thẻ.


- Các nhóm biến chính:


  - Numerical: `Customer_Age`, `Months_on_book`, `Credit_Limit`, `Total_Trans_Amt`, `Total_Trans_Ct`, `Avg_Utilization_Ratio`,...


  - Categorical: `Gender`, `Education_Level`, `Marital_Status`, `Income_Category`, `Card_Category`.


- Mục tiêu phân tích: khám phá phân bố, xác định đặc trưng quan trọng ảnh hưởng churn, chuẩn bị cho mô hình dự báo `Attrition_Flag`.


- Thách thức: lớp churn chiếm tỷ lệ nhỏ (~16%), cần lưu ý khi xây dựng mô hình và đánh giá hiệu năng.




-> Tóm lại, đây là bộ dữ liệu thực hành kinh điển cho bài toán churn trong lĩnh vực ngân hàng/thẻ tín dụng, thích hợp để minh họa pipeline EDA → tiền xử lý → mô hình phân loại và các khuyến nghị giữ chân khách hàng.

## II. DATA EXPLORATION

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats

### Dataset overview

#### Basic Information

##### Đọc dữ liệu

In [ ]:
data_path = "data/BankChurners.csv"
df = pd.read_csv(data_path)
df.head()

##### Số dòng và cột

In [ ]:
rows, cols = df.shape
print(f"Số dòng: {rows}, Số cột: {cols}")

- Như vậy data có số dòng là 10.127 và số cột là 23.

##### Các dòng dữ liệu
Mỗi dòng có ý nghĩa gì? Có dòng nào có ý nghĩa khác với các dòng còn lại không?

In [ ]:
df.sample(10)

- Mỗi dòng tương ứng một khách hàng thẻ tín dụng, có vẻ không có dòng nào khác biệt bất thường.

##### Overall size

In [ ]:
size_mb = os.path.getsize("data/BankChurners.csv") / (1024**2)
print(f"Dataset size: {size_mb:.2f} MB")

#### Data Integrity

##### Kiểm tra trùng lặp

In [ ]:
# Kiểm tra trùng lặp hoàn toàn
duplicate_count = df.duplicated().sum()
print(f"Số dòng trùng lặp hoàn toàn: {duplicate_count}")
df[df.duplicated()]

- Không có dòng nào bị trùng hoàn toàn cả.

##### Kiểm tra có dòng nào "empty" hoàn toàn không

In [ ]:
empty_rows = df.isna().all(axis=1).sum()
print("Số dòng hoàn toàn trống:", empty_rows)

- Không có dòng nào hoàn toàn empty cả

#### Column Inventory

##### Ý nghĩa các cột

In [ ]:
list(df.columns.values)

| Cột | Ý nghĩa |
|---|---|
| CLIENTNUM | Định danh nội bộ của khách hàng trong hệ thống thẻ tín dụng. |
| Attrition_Flag | Trạng thái khách hàng: `Attrited Customer` (đã đóng) hoặc `Existing Customer` (còn hoạt động). |
| Customer_Age | Tuổi của khách hàng. |
| Gender | Giới tính (`M`/`F`). |
| Dependent_count | Số người phụ thuộc trong hộ gia đình. |
| Education_Level | Trình độ học vấn (High School, Graduate, Doctorate, Unknown,...). |
| Marital_Status | Tình trạng hôn nhân (Married, Single, Divorced,...). |
| Income_Category | Nhóm thu nhập hằng năm (ví dụ: `< $40K`, `$60K - $80K`). |
| Card_Category | Hạng thẻ tín dụng (Blue, Silver, Gold, Platinum). |
| Months_on_book | Số tháng khách hàng đã gắn bó với ngân hàng (tuổi thẻ). |
| Total_Relationship_Count | Tổng số sản phẩm tài chính khách hàng sở hữu. |
| Months_Inactive_12_mon | Số tháng không hoạt động trong 12 tháng gần nhất. |
| Contacts_Count_12_mon | Số lần nhân viên liên hệ khách trong 12 tháng gần nhất. |
| Credit_Limit | Hạn mức tín dụng hiện tại. |
| Total_Revolving_Bal | Số dư quay vòng (revolving balance) trên thẻ. |
| Avg_Open_To_Buy | Hạn mức còn lại trung bình (Credit_Limit - Total_Revolving_Bal). |
| Total_Amt_Chng_Q4_Q1 | Mức thay đổi tổng giá trị giao dịch giữa quý 4 và quý 1. |
| Total_Trans_Amt | Tổng giá trị giao dịch thẻ trong 12 tháng gần nhất. |
| Total_Trans_Ct | Tổng số lượng giao dịch thẻ trong 12 tháng gần nhất. |
| Total_Ct_Chng_Q4_Q1 | Mức thay đổi số lượng giao dịch giữa quý 4 và quý 1. |
| Avg_Utilization_Ratio | Tỷ lệ sử dụng tín dụng trung bình (Total_Revolving_Bal / Credit_Limit). |
| Naive_Bayes_Classifier_..._Education_Level_Months_Inactive_12_mon_1 | Xác suất từ mô hình Naive Bayes nội bộ (thành phần 1) hỗ trợ phân loại churn. |
| Naive_Bayes_Classifier_..._Education_Level_Months_Inactive_12_mon_2 | Xác suất từ mô hình Naive Bayes nội bộ (thành phần 2) bổ sung cho biến trên. |

##### Những cột có tiềm năng để phân tích chuyên sâu
Các cột về đặc tính tài khoản (Account / Relationship)

- Months_on_book

- Total_Relationship_Count

- Card_Category

- Credit_Limit

- Avg_Open_To_Buy

- Avg_Utilization_Ratio

##### Drop Columns
- Xóa cột "CLIENTNUM" vì cột này không mang thông tin để phân tích
- Xóa 2 cột cuối cùng "Naive_Bayes_Classifier..." vì 2 cột này không phải dữ liệu thực tế, được khuyến cáo nên xóa trên kaggle

In [ ]:
df = df.drop(columns=['CLIENTNUM'])
df = df.drop(columns=df.columns[-2:])

#### Data Types

##### Kiểu dữ liệu của từng cột

In [ ]:
df.dtypes

##### Có cột nào có kiểu dữ liệu không thích hợp không?
- Nhìn chung thì các cột đều có dữ liệu phù hợp

##### Các cột cần convert data type
- Các cột đều có kiểu dữ liệu phù hợp nên không cột nào cần convert

In [ ]:
df.info()

- Như vậy các cột dữ liệu đã ổn để tiếp tục.

### Numerical Columns Analysis

Trước hết, ta cần filter các cột numerical

In [ ]:
numerical_cols = df.select_dtypes(include=[np.number]).columns
numerical_cols

#### Phân phối dữ liệu và khuynh hướng trung tâm    

##### Hình dạng phân phối

In [ ]:
skewness = df[numerical_cols].skew()
for col in numerical_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col], kde=True)
    plt.title(f"Histogram & Density: {col}")
    plt.show()

    print(f"\nColumn: {col}")
    print(f"Độ lệch: {skewness[col]}")

    if skewness[col] > 0.5:
        print("Dữ liệu lệch phải\n")
    elif skewness[col] < -0.5:
        print("Dữ liệu lệch trái\n")
    else:
        print("Dữ liệu cân bằng")


- Có nhiều biến như Customer_Age hay Dependent_count có phân phối gần chuẩn, tức là khá đối xứng và ổn định. 

- Trong khi một số biến tài chính như Credit_Limit, Total_Trans_Amt và Avg_Utilization_Ratio bị lệch phải, nghĩa là phần lớn khách hàng có giá trị thấp, chỉ một nhóm nhỏ có giá trị rất cao. 

- Điều này có thể cho chúng ta gợi ý rằng hành vi chi tiêu và sử dụng thẻ của khách hàng không đồng đều và có thể sử dụng yếu tố này để dự đoán nguy cơ rời bỏ.

#### Range & Outliers
Ở phần này, với các cột có kiểu dữ liệu số để hiểu thêm về dữ liệu, ta sẽ tính:
- Số các giá trị thiếu
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu 
- Giá trị mean
- Giá trị min
- Giá trị median
- Giá trị std
- Giá trị max

In [ ]:
num_summary_rows = []
for col in numerical_cols:
    s = df[col]
    mean_val = s.mean()
    std_val = s.std()
    min_val = s.min()
    q1 = s.quantile(0.25)
    median_val = s.median()
    q3 = s.quantile(0.75)
    max_val = s.max()
    num_summary_rows.append({
        'column': col,
        'mean': mean_val,
        'std': std_val,
        'min': min_val,
        'q1': q1,
        'median': median_val,
        'q3': q3,
        'max': max_val
    })
num_summary_df = pd.DataFrame(num_summary_rows)
num_summary_df

##### Outliers Detection
Chúng ta tiến hành kiểm tra xem các cột dữ liệu số có xuất hiện giá trị ngoại lai (outliers) hay không bằng kỹ thuật IQR outlier detection.

In [ ]:
outlier_rows = []
outliers = []
for col in numerical_cols:
    s = df[col].dropna()
    q1, q3 = s.quantile([0.25,0.75])
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    mask = (s < lower) | (s > upper)
    outliers.append(s[mask].values)
    outlier_rows.append({"column": col, "outliers": mask.sum(), "percent": round(mask.sum()/len(s)*100,2)})
pd.DataFrame(outlier_rows)

##### Kiểm tra outliers

In [ ]:
outlier_rows = []

for i in range(len(numerical_cols)):
    print(f"Outliers in {numerical_cols[i]}: {outliers[i][:10]}")


- Nhận xét: Các giá trị outliers được in ra hoàn toàn có thể xảy ra, không quá bất thường nên các outliers này là "genuine extreme values"

#### Data Quality

In [ ]:
missing_summary_rows = []
for col in numerical_cols:
    s = df[col]
    missing = s.isna().sum()
    missing_pct = round(missing/len(s)*100,2)
    missing_summary_rows.append({
        'column': col,
        'missing': missing,
        'missing_%': missing_pct
    })
missing_summary_df = pd.DataFrame(missing_summary_rows)
missing_summary_df

Nhận xét: Tất cả các cột đều đây đủ, không chứa missing values nào. Lý giải một chút, một số tổ chức ngân hàng có quy trình ETL chuẩn:

- Missing được xử lý trước khi release dataset

- Các trường bắt buộc đã có giá trị mặc định

Vậy nên dataset này không chứa missing values nào thì cũng là điều dễ hiểu

### Categorical Columns Analysis

Tương tự như numerical columns, ta cũng lọc các cột categorical

In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns
categorical_cols

#### Value Distribution

##### Unique values

In [ ]:
summary = []

for col in categorical_cols:
    s = df[col]

    # 1. Số lượng giá trị distinct
    unique_count = s.nunique()

    # 2. Top 10 giá trị frequent
    top_values = s.value_counts().head(10)

    summary.append({
        "column": col,
        "unique_values": unique_count,
        "top_values": top_values.to_dict()
    })

    # 3. Visualization
    plt.figure(figsize=(10, 4))
    sns.countplot(x=s, order=s.value_counts().index)
    plt.title(f"Value Distribution - {col}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    print(f"\nColumn: {col}")
    print(f"Unique values: {unique_count}")
    print(f"Top values:\n{top_values}")

    max_ratio = top_values.iloc[0] / len(s)
    if max_ratio > 0.6:
        print("Dữ liệu không cân bằng\n")
    else:
        print("Dữ liệu cân bằng\n")

pd.DataFrame(summary)


Nhìn nhanh vào các biến phân loại cho thấy khách hàng còn sử dụng thẻ (Existing) chiếm đa số (~84%), hạng thẻ Blue áp đảo nên cần cân nhắc kỹ khi trực quan hoặc xây dựng mô hình.

#### Data Quality

Ở đây hơi khác một chút so với numerical cols thì categorical cols có sự xuất hiện của missing values. Cụ thể ở các cột Educational_level, Marital_status, Income_Category có các giá trị "Unknown". Ta có thể xem đây là missing values.

##### Tỉ lệ missing values

In [ ]:
for col in categorical_cols:
    missing = len(df[df[col] == "Unknown"])
    missing_pct = round(missing/len(df)*100,2)
    print(f"Column: {col}, Missing: {missing}, Missing %: {missing_pct}%")


##### Kiểm tra giá trị có bị inconsistencied không
Theo quan ở trên, các giá trị đều nhất quán, không có cái nào bị trùng về mặt ý nghĩa

##### Category quá ít quan sát

In [ ]:
rare_threshold = 0.01
rare_categories = {}

for col in categorical_cols:
    freq = df[col].value_counts(normalize=True)
    rare = freq[freq < rare_threshold]
    if len(rare) > 0:
        rare_categories[col] = rare

for col, rare in rare_categories.items():
    print(f"\nColumn: {col}")
    print(rare)

- Nhận xét: ta thấy trong cột Card_Category thì giá trị Platinum chiếm chưa đến 0.2%, quá ít. Giá trị này có thể xem xét gộp chung với giá trị khác

### Missing Data Analysis

Trước hết ta chuyển các giá trị "Unknown" về nan 

In [ ]:
df.replace("Unknown", np.nan, inplace=True)

#### Overall Assessment

##### Missing values Summary

In [ ]:
missing_summary = []
for col in df.columns:
    missing = df[col].isna().sum()
    missing_pct = round(missing/len(df)*100,2)
    missing_summary.append({
        'column': col,
        'missing': missing,
        'missing_pct': missing_pct
    })
missing_summary_df = pd.DataFrame(missing_summary)
missing_summary_df


- Ta có thể thấy, trong tổng số 20 cột dữ liệu, chỉ có 3 cột chứa giá trị thiếu (missing values) gồm Education_Level, Marital_Status và Income_Category. Các cột còn lại đều không có giá trị thiếu, cho thấy bộ dữ liệu tương đối sạch và đã được xử lý tốt ở bước tiền xử lý.

- Tuy nhiên, tỷ lệ missing values ở ba cột này là khá đáng kể, trong đó Education_Level có tỷ lệ cao nhất (15%), tiếp theo là Income_Category (10.98%) và Marital_Status (7.40%). Đây đều là các biến phân loại quan trọng, do đó cần lựa chọn phương pháp xử lý phù hợp.

##### Visualize missing data patterns

In [ ]:
plt.figure()
sns.barplot(x='missing_pct', y='column', data=missing_summary_df)
plt.xlabel('Tỷ lệ missing (%)')
plt.ylabel('Cột')
plt.title('Tỷ lệ giá trị missing')
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(df.isna(), cbar=False, cmap='viridis')
plt.title('Pattern missing trong dataset')
plt.show()

- Nhận xét: các missing values phân bố hoàn toàn ngẫu nhiên, không có pattern cụ thể

### Relationships & Correlations

#### Preliminary Patterns

##### Calculate correlation matrix for numerical variables

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
corr_matrix = df[numerical_cols].corr() 
print(corr_matrix.head())

##### Correlation heatmap

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Correlation Heatmap")
plt.show()

##### Identify strongly correlated pairs

In [ ]:
strong_corrs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_value = corr_matrix.iloc[i,j]
        if abs(corr_value) >= 0.7:
            strong_corrs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_value))

strong_corrs_df = pd.DataFrame(strong_corrs, columns=['Column1', 'Column2', 'Correlation'])
print(strong_corrs_df)

Nhận xét: 
- Customer_Age vs Months_on_book: Không quá bất ngờ, vì tuổi tác thường liên quan đến độ trung thành hoặc thời gian sử dụng dịch vụ
- Credit_Limit vs Avg_Open_To_Buy: Đây là mối quan hệ công thức, không phải thực tế hành vi khách hàng
- Total_Trans_Amt vs Total_Trans_Ct: Hợp lý, nhưng cũng có thể cần kiểm tra giao dịch trung bình để xem có khách hàng nào chi tiêu lớn chỉ với ít giao dịch hay không.

#### Cross-tabulations

##### Categorical × categorical combinations

Ta sẽ kiểm định xem một điều hay được nói trong cuộc sống: 'Học càng cao, lương càng cao'

In [ ]:
crosstab = pd.crosstab(df['Education_Level'], df['Income_Category'], margins=True, normalize='index')
crosstab

- Ta có thấy một kết quả khá bất ngờ và phản trực giác so với suy nghĩ thông thường là "Học càng cao, lương càng cao".
    - Nhóm Doctorate (Tiến sĩ): Là nhóm có tỷ lệ thu nhập cao ($120K+) lớn nhất (9.7%). Tuy nhiên, họ cũng chính là nhóm có tỷ lệ thu nhập thấp (Less than $40K) cao nhất (41.5%). Điều này cho thấy sự phân hóa cực cao trong nhóm học vấn cao nhất.
    - Nhóm Uneducated (Không bằng cấp): Rất bất ngờ, đây là nhóm có tỷ lệ người thu nhập cao ($120K+) đứng thứ 2 (9.1%), chỉ thua nhóm Tiến sĩ một chút và cao hơn cả Cử nhân hay Thạc sĩ.
    - Nhóm Post-Graduate (Sau ĐH): Lại là nhóm có tỷ lệ người thu nhập cao ($120K+) thấp nhất bảng (6.4%), thấp hơn cả nhóm không có bằng cấp.
    
-> Cột Income_category và cột Education_Leval không phải là sự lựa chọn tốt, giữa chúng chưa thấy sự tỉ lệ nào ổn định cả

##### Numerical × categorical combinations

Ta kiểm tra thử liệu những khách hàng sắp bỏ đi thì sẽ có tổng giá trị giao dịch thấp hơn nhóm người ở lại không?

In [ ]:
df.groupby('Attrition_Flag')['Total_Trans_Amt'].describe()

- Dễ dàng thấy được sự chênh lệch rõ rệt: Khách hàng rời bỏ (Attrited Customer) có tổng số tiền giao dịch thấp hơn hẳn so với khách hàng hiện tại (Existing Customer). Trung bình nhóm Attrited chỉ tiêu khoảng 3,095, trong khi nhóm Existing là 4,654 (cao hơn ~50%).
- Phân phối: 75% số khách hàng rời bỏ có tổng giao dịch dưới 2,772, con số này chỉ nhỉnh hơn mức 25% thấp nhất của nhóm khách hàng hiện tại (2,384) một chút.

-> Cột 'Total_Trans_Amt' là một biến rất quan trọng để dự đoán việc khách hàng rời bỏ. Khách tiêu càng ít, nguy cơ rời bỏ càng cao.

Ta xem thử xu hướng thay đổi tổng số lần giao dịch Q4 so với Q1 có phải là dấu hiệu dự đoán khách hàng bỏ đi hay không?

In [ ]:
df.groupby('Attrition_Flag')['Total_Ct_Chng_Q4_Q1'].describe()


- Nhóm rời bỏ (Attrited) có chỉ số trung bình chỉ là 0.55. Tức là vào Q4 (cuối năm), họ chỉ quẹt thẻ bằng khoảng một nửa số lần so với Q1 (đầu năm).
- Nhóm ở lại (Existing) có chỉ số là 0.74. Họ cũng giảm tần suất, nhưng mức giảm nhẹ hơn nhiều.

-> Sự sụt giảm đột ngột về tần suất sử dụng (số lần quẹt thẻ) là một tín hiệu cho dự đoán khách hàng rời đi

### Initial Observations & Insights

##### Một số observation
Như vậy là quá trình EDA dữ liệu, ta đã có được một vài nhận xét, hiểu biết về dữ liệu, một số observation chính:
- Tổng quan:
    - Dữ liệu có hơn 10000 dòng, không có dòng nào trùng lặp hay bị empty
    - Dữ liệu đã được xử lý khá sạch, các giá trị Nan đã được xử lý về giá trị 'Unknown'.
    - Dữ liệu có 20 cột, 6 cột categorical, 14 cột numerical, các cột đều có dữ liệu thích hợp
- Các cột categorical:
    - Chứa các giá trị Nan đã được xử lý về giá trị 'Unknown'
    - Phân phối dữ liệu đang khá cân bằng, một số cột bị lệch phải
    - Một vài cột bị mất cân bằng nặng(Card_Category, Attrited_Flag) cần tiền xử lý nếu sử dụng mô hình
- Các cột numerical:
    - Không chứa giá trị Nan nào
    - Một số cột chứa Outliers nhưng không quá bất thường

##### Một số vấn đề về data quality
- Data đang bị mất cân bằng ở cột "Attrition"
- Data đang bị missing value tại các cột categorical: Education_Level, Marital_Status, Income_Category

##### Các bước tiền xử lý cần có

- Xử lý imbalance
- Xử lý missing values